### This script performs a varimax-rotated PCA (R-verified)

In [ ]:
%load_ext rpy2.ipython
%config InlineBackend.figure_format = 'retina'

import pandas as pd
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#### set directory and import dataset

In [ ]:
datadir = '/home/kwesi/terra/research/CMIP5_models/cmip5_smhi/cmip_trbi/roll_freq_matrix/'

In [ ]:
data = pd.read_csv(datadir + 'zg700_ECMWF-ERAINT_rcp85_r1i1p1_19800101-20131231_matrix.csv',names = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO'], header =0)

#### select specific columns in dataset

In [ ]:
df = data.iloc[:,0:15]

In [ ]:
type(df); df.head()

#### extract values at each node of the SOM frequency dataset

In [ ]:
nodes = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO']

## Separating out the nodes
x = df.loc[:, nodes].values

# Standardizing the nodes
x = StandardScaler().fit_transform(x)
#%store x

#### perform factor analysis with varimax rotation

In [ ]:
n_comps = 3

In [ ]:
fan = FactorAnalysis(n_components=n_comps, rotation='varimax').fit(x)
#fan.get_covariance().shape

fan.components_.T[:,0:3];

#get scores of pca
fan_scores = fan.transform(x)

#calculate explained sample variance
n_sample = x.shape[1]
exp_v = np.array([(fan.components_.T[:, i]**2).sum()/(n_sample) for i in range(n_comps)]).round(4)
print(100*exp_v)

#var = np.sum(fan.components_.T**2, axis=0) #coulmn-wise
#print(var)
#exp_v = (100*(var/np.sum(var))[0:3])
#print(exp_v)
#display(pd.DataFrame(fan_scores[:,0:3]));
#fan.get_covariance

In [ ]:
#plt.bar(range(15),fan.components_[0,:]**2

In [ ]:
df_scores = pd.DataFrame(fan_scores[:,0:3],columns = ['RC1', 'RC2','RC3'])
df_scores;
#df_scores.to_csv(datadir + 'ERAINT_pca_scores.csv')

#### create a dataframe from rotated principal compnents

In [ ]:
principalDf = pd.DataFrame(data = fan.components_.T[:,0:3], columns = ['RC1', 'RC2','RC3'])

In [ ]:
principalDf.rename(index={0:'0,0',1:'0,1',2:'0,2',3:'1,0',4:'1,1',5:'1,2',
                         6:'2,0',7:'2,1',8:'2,2',9:'3,0',10:'3,1',11:'3,2',
                         12:'ITCZInt',13:'NINO34',14:'AAO'}, inplace=True)


In [ ]:
principalDf;

#### fitting CMIP5 model data to observational data for scores

In [ ]:
CanESM = pd.read_csv(datadir + 'zg700_CanESM2_rcp85_r1i1p1_19800101-20131231_matrix.csv',names = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO'], header =0)
#zg700_CanESM2_rcp85_r1i1p1_19800101-20131231_matrix.csv
CanESM_df = CanESM.iloc[:,0:15]
CanESM_df.head();


#### extract values at each node of the SOM frequency dataset
CanESM_nodes = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO']

## Separating out the nodes
CanESM_x = CanESM_df.loc[:, CanESM_nodes].values

# Standardizing the nodes and project to my data
CanESM_x = StandardScaler().fit_transform(CanESM_x)
#%store CanESM_x

#use era interim pca to predict model pca
CanESM_scores_x = fan.transform(CanESM_x)

#calculate explained sample variance
n_sample = x.shape[1]
CanESM_var = np.array([(CanESM_scores_x[:, i]**2).sum()/(n_sample-1) for i in range(n_comps)]).round(2)
print(CanESM_var[0:3])


#var = np.sum(CanESM_scores_x.T**2, axis=1) #coulmn-wise
#CanESM_var = 100 * var/np.sum(var)
#print(CanESM_var[0:3])

CanESM_scores = pd.DataFrame(CanESM_scores_x[:,0:3],columns = ['RC1', 'RC2','RC3'])
CanESM_scores;
#CanESM_scores.to_csv(datadir + 'CanESM_pca_scores.csv')


In [ ]:
CNRM = pd.read_csv(datadir + 'zg700_CNRM-CM5_rcp85_r1i1p1_19800101-20131231_matrix.csv',names = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO'], header =0)

CNRM_df = CNRM.iloc[:,0:15]
#print(CNRM_df.head())


#### extract values at each node of the SOM frequency dataset
CNRM_nodes = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO']

## Separating out the nodes
CNRM_x = CNRM_df.loc[:, CNRM_nodes].values

# Standardizing the nodes
CNRM_x = StandardScaler().fit_transform(CNRM_x)
#%store CNRM_x

#use era interim pca to predict model pca
CNRM_scores_x = fan.transform(CNRM_x)

#calculate explained variance
#n_sample = x.shape[1]
CNRM_var = np.array([(CNRM_scores_x[:, i]**2).sum()/(n_sample-1) for i in range(n_comps)]).round(2)
print(CNRM_var[0:3])


CNRM_scores_x;

CNRM_scores = pd.DataFrame(CNRM_scores_x[:,0:3],columns = ['RC1', 'RC2','RC3'])
CNRM_scores;
#CNRM_scores.to_csv(datadir +'CNRM_pca_scores.csv')


In [ ]:
GFDL = pd.read_csv(datadir + 'zg700_GFDL-ESM2M_rcp85_r1i1p1_19800101-20131231_matrix.csv',names = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO'], header =0)

GFDL_df = GFDL.iloc[:,0:15]
GFDL_df.head();


#### extract values at each node of the SOM frequency dataset
GFDL_nodes = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO']

## Separating out the nodes
GFDL_x = GFDL_df.loc[:, GFDL_nodes].values

# Standardizing the nodes
GFDL_x = StandardScaler().fit_transform(GFDL_x)
#%store GFDL_x

#use era interim pca to predict model pca
GFDL_scores_x = fan.transform(GFDL_x)

#calculate explained variance
n_sample = x.shape[1]
GFDL_var = np.array([(GFDL_scores_x[:, i]**2).sum()/(n_sample-1) for i in range(n_comps)]).round(2)
print(GFDL_var[0:3])

GFDL_scores = pd.DataFrame(GFDL_scores_x[:,0:3],columns = ['RC1', 'RC2','RC3'])
GFDL_scores;
#GFDL_scores.to_csv(datadir + 'GFDL_pca_scores.csv')


In [ ]:
HadGEM = pd.read_csv(datadir + 'zg700_HadGEM2-ES_rcp85_r1i1p1_19800101-20131231_matrix.csv',names = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO'], header =0)

HadGEM_df = HadGEM.iloc[:,0:15]
HadGEM_df.head();


#### extract values at each node of the SOM frequency dataset
HadGEM_nodes = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO']

## Separating out the nodes
HadGEM_x = HadGEM_df.loc[:, HadGEM_nodes].values

# Standardizing the nodes
HadGEM_x = StandardScaler().fit_transform(HadGEM_x)
#%store HadGEM_x

#use era interim pca to predict model pca
HadGEM_scores_x = fan.transform(HadGEM_x)

#calculate explained variance
n_sample = x.shape[1]
HadGEM_var = np.array([(HadGEM_scores_x[:, i]**2).sum()/(n_sample-1) for i in range(n_comps)]).round(2)
print(HadGEM_var[0:3])

HadGEM_scores = pd.DataFrame(HadGEM_scores_x[:,0:3],columns = ['RC1', 'RC2','RC3'])
HadGEM_scores;
#HadGEM_scores.to_csv(datadir + 'HadGEM_pca_scores.csv')


In [ ]:
IPSL = pd.read_csv(datadir + 'zg700_IPSL-CM5A-LR_rcp85_r1i1p1_19800101-20131231_matrix.csv',names = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO'], header =0)

IPSL_df = IPSL.iloc[:,0:15]
IPSL_df.head();


#### extract values at each node of the SOM frequency dataset
IPSL_nodes = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO']

## Separating out the nodes
IPSL_x = IPSL_df.loc[:, IPSL_nodes].values

# Standardizing the nodes
IPSL_x = StandardScaler().fit_transform(IPSL_x)
#%store IPSL_x

#use era interim pca to predict model pca
IPSL_scores_x = fan.transform(IPSL_x)

#calculate explained variance
n_sample = x.shape[1]
IPSL_var = np.array([(IPSL_scores_x[:, i]**2).sum()/(n_sample-1) for i in range(n_comps)]).round(2)
print(IPSL_var)

IPSL_scores = pd.DataFrame(IPSL_scores_x[:,0:3],columns = ['RC1', 'RC2','RC3'])
IPSL_scores;
#IPSL_scores.to_csv(datadir + 'IPSL_pca_scores.csv')


In [ ]:
MIROC = pd.read_csv(datadir + 'zg700_MIROC-ESM_rcp85_r1i1p1_19800101-20131231_matrix.csv',names = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO'], header =0)

MIROC_df = MIROC.iloc[:,0:15]
MIROC_df.head();


#### extract values at each node of the SOM frequency dataset
MIROC_nodes = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO']

## Separating out the nodes
MIROC_x = MIROC_df.loc[:, MIROC_nodes].values

# Standardizing the nodes
MIROC_x = StandardScaler().fit_transform(MIROC_x)
#%store MIROC_x

#use era interim pca to predict model pca
MIROC_scores_x = fan.transform(MIROC_x)

#calculate explained variance
n_sample = x.shape[1]
MIROC_var = np.array([(MIROC_scores_x[:, i]**2).sum()/(n_sample-1) for i in range(n_comps)]).round(2)
print(MIROC_var)

MIROC_scores = pd.DataFrame(MIROC_scores_x[:,0:3],columns = ['RC1', 'RC2','RC3'])
MIROC_scores;
#MIROC_scores.to_csv(datadir + 'MIROC_pca_scores.csv')


In [ ]:
MPI = pd.read_csv(datadir + 'zg700_MPI-ESM-LR_rcp85_r1i1p1_19800101-20131231_matrix.csv',names = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO'], header =0)

MPI_df = MPI.iloc[:,0:15]
MPI_df.head();


#### extract values at each node of the SOM frequency dataset
MPI_nodes = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO']

## Separating out the nodes
MPI_x = MPI_df.loc[:, MPI_nodes].values

# Standardizing the nodes
MPI_x = StandardScaler().fit_transform(MPI_x)
#store MPI_x

#use era interim pca to predict model pca
MPI_scores_x = fan.transform(MPI_x)

#calculate explained variance
n_sample = x.shape[1]
MPI_var = np.array([(MPI_scores_x[:, i]**2).sum()/(n_sample-1) for i in range(n_comps)]).round(2)
print(MPI_var[0:3])

MPI_scores = pd.DataFrame( MPI_scores_x[:,0:3],columns = ['RC1', 'RC2','RC3'])
MPI_scores;
#MPI_scores.to_csv(datadir +'MPI_pca_scores.csv')


In [ ]:
MRI = pd.read_csv(datadir + 'zg700_MRI-CGCM3_rcp85_r1i1p1_19800101-20131231_matrix.csv',names = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO'], header =0)

MRI_df = MRI.iloc[:,0:15]
MRI_df.head();


#### extract values at each node of the SOM frequency dataset
MRI_nodes = ['0,0','0,1','0,2','1,0','1,1','1,2','2,0','2,1','2,2','3,0','3,1'
         ,'3,2','ITCZInt','NINO34','AAO']

## Separating out the nodes
MRI_x = MRI_df.loc[:, MRI_nodes].values

# Standardizing the nodes
MRI_x = StandardScaler().fit_transform(MRI_x)
#%store MRI_x

#use era interim pca to predict model pca
MRI_scores_x = fan.transform(MRI_x)

#calculate sample variance
n_sample = x.shape[1]
MRI_var = np.array([(MRI_scores_x[:, i]**2).sum()/(n_sample-1) for i in range(n_comps)]).round(2)
print(MRI_var[0:3])

MRI_scores = pd.DataFrame(MRI_scores_x[:,0:3],columns = ['RC1', 'RC2','RC3'])
MRI_scores;
#MRI_scores.to_csv(datadir + 'MRI_pca_scores.csv')


In [ ]:
#explained_variance = {'ERAInterim' :[30.0, 19.80, 12.60], 'CanESM':[17.41, 21.65, 20.78],'CNRM':[15.5, 23.88, 21.56], 
#                      'GFDL':[18.5, 21.17, 19.13], 'HadGEM':[20.21, 18.11, 17.74], 'IPSL':[17.66, 19.09, 23.56], 
#                      'MIROC':[19.98, 18.82, 21.30], 'MPI':[20.92, 19.69, 16.65], 'MRI':[16.81, 23.25, 19.35]}

In [ ]:
#exp_var = pd.DataFrame(explained_variance)
#print(exp_var.T)
#exp_var.T.to_csv(datadir + 'explained_variance_gcms_future_NEW.csv')

In [ ]:
#ts=pd.DataFrame()
tr = np.var(CNRM_df.iloc[:,12])
#ts['var'] = tr.var()
print(tr)

In [ ]:
#ts=pd.DataFrame()
#tr = df.iloc[:,12].var(axis=0)
#ts['var'] = tr.var()
#print(tr)
plt.rcParams['figure.figsize'] = [8, 6]
#plt.rcParams['font.size'] = 15

era = df.iloc[:,12]
era.reset_index(drop=True, inplace=True)
eraa = (era-np.mean(era))/np.std(era)


can = CanESM_df.iloc[:,12]
can.reset_index(drop=True, inplace=True)
cana = (can-np.mean(can))/np.std(can)

#cnr = CNRM_df.iloc[:,12]
#cnr.to_csv('test_cnrm.csv')
cnr = pd.read_csv('test_cnrm1.csv')
cnrb = cnr.iloc[:,1]
cnrb.reset_index(drop=True, inplace=True)
cnra = (cnrb-np.mean(cnrb))/np.std(cnrb)


gfd = GFDL_df.iloc[:,12]
gfd.reset_index(drop=True, inplace=True)
gfda = (gfd-np.mean(gfd))/np.std(gfd)


had = HadGEM_df.iloc[:,12]
had.reset_index(drop=True, inplace=True)
hada = (had-np.mean(had))/np.std(had)


ips = IPSL_df.iloc[:,12]
ips.reset_index(drop=True, inplace=True)
ipsa = (ips-np.mean(ips))/np.std(ips)


mir = MIROC_df.iloc[:,12]
mir.reset_index(drop=True, inplace=True)
mira = (mir-np.mean(mir))/np.std(mir)


mpi = MPI_df.iloc[:,12]
mpi.reset_index(drop=True, inplace=True)
mpia = (mpi-np.mean(mpi))/np.std(mpi)


mri = MRI_df.iloc[:,12]
mri.reset_index(drop=True, inplace=True)
mria = (mri-np.mean(mri))/np.std(mri)


era2 = df.iloc[:,13]
era2.reset_index(drop=True, inplace=True)
#print(ts)

can2 = CanESM_df.iloc[:,13]
can2.reset_index(drop=True, inplace=True)

cnr2 = CNRM_df.iloc[:,13]
cnr2.reset_index(drop=True, inplace=True)

gfd2 = GFDL_df.iloc[:,13]
gfd2.reset_index(drop=True, inplace=True)


had2 = HadGEM_df.iloc[:,13]
had2.reset_index(drop=True, inplace=True)


ips2 = IPSL_df.iloc[:,13]
ips2.reset_index(drop=True, inplace=True)


mir2 = MIROC_df.iloc[:,13]
mir2.reset_index(drop=True, inplace=True)


mpi2 = MPI_df.iloc[:,13]
mpi2.reset_index(drop=True, inplace=True)


mri2 = MRI_df.iloc[:,13]
mri2.reset_index(drop=True, inplace=True)



era3 = df.iloc[:,14]
era3.reset_index(drop=True, inplace=True)
#print(ts)

can3 = CanESM_df.iloc[:,14]
can3.reset_index(drop=True, inplace=True)

cnr3 = CNRM_df.iloc[:,14]
cnr3.reset_index(drop=True, inplace=True)

gfd3 = GFDL_df.iloc[:,14]
gfd3.reset_index(drop=True, inplace=True)


had3 = HadGEM_df.iloc[:,14]
had3.reset_index(drop=True, inplace=True)


ips3 = IPSL_df.iloc[:,14]
ips3.reset_index(drop=True, inplace=True)


mir3 = MIROC_df.iloc[:,14]
mir3.reset_index(drop=True, inplace=True)


mpi3 = MPI_df.iloc[:,14]
mpi3.reset_index(drop=True, inplace=True)


mri3 = MRI_df.iloc[:,14]
mri3.reset_index(drop=True, inplace=True)


In [ ]:
plt.plot(eraa)
plt.plot(era2)
plt.plot(era3)
plt.title('ERA-Interim')
plt.legend()
plt.show()

plt.plot(cana)
plt.plot(can2)
plt.plot(can3)
plt.title('CanESM')
plt.legend()
plt.show()



plt.plot(cnra)
plt.plot(cnr2)
plt.plot(cnr3)
plt.title('CNRM')
plt.legend()
plt.show()



plt.plot(gfda)
plt.plot(gfd2)
plt.plot(gfd3)
plt.title('GFDL')
plt.legend()
plt.show()



plt.plot(hada)
plt.plot(had2)
plt.plot(had3)
plt.title('HadGEM2')
plt.legend()
plt.show()



plt.plot(ipsa)
plt.plot(ips2)
plt.plot(ips3)
plt.title('IPSL')
plt.legend()
plt.show()



plt.plot(mira)
plt.plot(mir2)
plt.plot(mir3)
plt.title('MIROC')
plt.legend()
plt.show()



plt.plot(mpia)
plt.plot(mpi2)
plt.plot(mpi3)
plt.title('MPI')
plt.legend()
plt.show()



plt.plot(mria)
plt.plot(mri2)
plt.plot(mri3)
plt.title('MRI')
plt.legend()
plt.show()

In [ ]:
era1 = era-np.mean(era)

can1 = can-np.mean(can)

cnr1 = cnrb-np.mean(cnrb)

gfd1 = gfd-np.mean(gfd)

had1 = had-np.mean(had)

ips1 = ips-np.mean(ips)

mir1 = mir-np.mean(mir)

mpi1 = (mpi-np.mean(mpi))

mri1 = (mri-np.mean(mri))

#print(era1)

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.colors as colors


tr = [[era1], [can1], [cnr1], [gfd1], [had1], [ips1], [mir1], [mpi1], [mri1]]
ni = [[era2], [can2], [cnr2], [gfd2], [had2], [ips2], [mir2], [mpi2], [mri2]]
aa = [[era3], [can3], [cnr3], [gfd3], [had3], [ips3], [mir3], [mpi3], [mri3]]

trbi = []
for i in tr:
    trb = np.var(i)
    trbi.append(trb)    
print(trbi)


nino = []
for j in ni:
    nin = np.var(j)
    nino.append(nin)    
print(nino)


aao = []
for k in aa:
    a = np.var(k)
    aao.append(a)    
print(aao)

### sample variance for each process index

In [ ]:
ERAInterimv = [0.65, 0.81, 0.48]
CanESMv = [2.94, 0.97, 0.46]
CNRMv  =  [2.20, 0.56, 0.46]
GFDLv  =  [4.68, 1.37, 0.45]
HadGEMv = [4.16, 0.83, 0.53]
IPSLv  =  [5.07, 1.21, 0.38]
MIROCv =  [1.90, 0.78, 0.55]
MPIv   =  [1.17, 1.29, 0.48]
MRIv   =  [1.29, 1.01, 0.41]

x = np.arange(len(CanESMv))  # the label locations
width = 0.1  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 8))
#ax = fig.add_subplot(111)
rects0 = ax.bar(x - width/2, ERAInterimv, width,edgecolor='black',label='ERA-Interim')
rects1 = ax.bar(x + width/2, CanESMv, width,edgecolor='black',label='CanESM2')
rects2 = ax.bar(x + 3*width/2, CNRMv, width, edgecolor='black',label='CNRM-CM5')
rects3 = ax.bar(x + 5*width/2, GFDLv, width, edgecolor='black', label='GFDL-ESM2M')
rects4 = ax.bar(x + 7*width/2, HadGEMv, width, edgecolor='black', label='HadGEM2-ES')
rects5 = ax.bar(x + 9*width/2, IPSLv, width, edgecolor='black', label='IPSL-CM5A-LR')
rects6 = ax.bar(x + 11*width/2, MIROCv, width, edgecolor='black', label='MIROC-ESM')
rects7 = ax.bar(x + 13*width/2, MPIv, width, edgecolor='black', label='MPI-ESM-LR')
rects8 = ax.bar(x + 15*width/2, MRIv, width, edgecolor='black', label='MRI-CGCM3')

# Fix the x-axes.
lab = ['TRBI', 'NINO34', 'AAO', 'residuals'] #['ERA-Interim','CanESM2','CNRM-CM5','GFDL-ESM2M', 'HadGEM2-ES', 'MIROC-ESM','MPI-ESM-LR','MRI-CGCM3']   
ax.set_xticks(x + 7*width / 2)
ax.set_xticklabels(lab,fontsize=10)
plt.ylabel('Sample Variance',fontsize=18)
ax.legend(fontsize=13)


# create a function
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom',fontsize=10)
autolabel(rects0)
autolabel(rects1)
autolabel(rects2)
autolabel(rects3)
autolabel(rects4)
autolabel(rects5)
autolabel(rects6)
autolabel(rects7)
autolabel(rects8)

fig.tight_layout()
#plt.savefig(datadir2 + 'sample_variance_gcms_hist_v1.png',bbox_inches='tight', dpi = 600) 
plt.show()

#### Plot explained variances for RCs

In [ ]:
datadir2 = '/terra/users/csag/kwesi/research/future/pca_matrices/' 

import matplotlib.pyplot as plt
import numpy as np



In [ ]:
ERAInterimh = [30.0, 19.8, 12.6, 37.6]
CanESMh = [19.9, 16.6, 22.7, 40.8]
CNRMh  =  [13.9, 23.3, 28.3, 34.5]
GFDLh  =  [17.3, 20.8, 22.6, 39.0]
HadGEMh = [20.1, 18.2, 22.9, 38.8]
IPSLh  =  [20.2, 18.5, 27.0, 37.3]
MIROCh =  [17.8, 24.3, 16.7, 41.2]
MPIh   =  [20.5, 17.9, 23.3, 38.3]
MRIh   =  [17.4, 22.8, 22.9, 36.9]

x = np.arange(len(CanESMh))  # the label locations
width = 0.1  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 8))
#ax = fig.add_subplot(111)
rects0 = ax.bar(x - width/2, ERAInterimh, width,edgecolor='black',label='ERA-Interim')
rects1 = ax.bar(x + width/2, CanESMh, width,edgecolor='black',label='CanESM2')
rects2 = ax.bar(x + 3*width/2, CNRMh, width, edgecolor='black',label='CNRM-CM5')
rects3 = ax.bar(x + 5*width/2, GFDLh, width, edgecolor='black', label='GFDL-ESM2M')
rects4 = ax.bar(x + 7*width/2, HadGEMh, width, edgecolor='black', label='HadGEM2-ES')
rects5 = ax.bar(x + 9*width/2, IPSLh, width, edgecolor='black', label='IPSL-CM5A-LR')
rects6 = ax.bar(x + 11*width/2, MIROCh, width, edgecolor='black', label='MIROC-ESM')
rects7 = ax.bar(x + 13*width/2, MPIh, width, edgecolor='black', label='MPI-ESM-LR')
rects8 = ax.bar(x + 15*width/2, MRIh, width, edgecolor='black', label='MRI-CGCM3')

# Fix the x-axes.
lab = ['PC1', 'PC2', 'PC3', 'residuals']
ax.set_xticks(x + 7*width / 2)
ax.set_xticklabels(lab,fontsize=18)
plt.ylabel('Explained Variance (%)',fontsize=18)
ax.legend(fontsize=10.5)


# create a function
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom',fontsize=8.5)
autolabel(rects0)
autolabel(rects1)
autolabel(rects2)
autolabel(rects3)
autolabel(rects4)
autolabel(rects5)
autolabel(rects6)
autolabel(rects7)
autolabel(rects8)

fig.tight_layout()
plt.savefig(datadir2 + 'explained_variance_gcms_hist_withresiduals_v1.png',bbox_inches='tight', dpi = 600) 
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ERAInterim = [37.6]
CanESM = [40.1]
CNRM  =  [39.0]
GFDL  =  [41.2]
HadGEM = [44.0]
IPSL  =  [39.5]
MIROC =  [40.0]
MPI   =  [42.7]
MRI   =  [40.5]


x = np.arange(len(CanESM))  # the label locations
width = 0.1  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 8))
rects0 = ax.bar(x - width/2, ERAInterim, width,edgecolor='black',label='ERA-Interim')
rects1 = ax.bar(x + width/2, CanESM, width,edgecolor='black',label='CanESM')
rects2 = ax.bar(x + 3*width/2, CNRM, width, edgecolor='black',label='CNRM')
rects3 = ax.bar(x + 5*width/2, GFDL, width, edgecolor='black', label='GFDL')
rects4 = ax.bar(x + 7*width/2, HadGEM, width, edgecolor='black', label='HadGEM')
rects5 = ax.bar(x + 9*width/2, IPSL, width, edgecolor='black', label='IPSL')
rects6 = ax.bar(x + 11*width/2, MIROC, width, edgecolor='black', label='MIROC')
rects7 = ax.bar(x + 13*width/2, MPI, width, edgecolor='black', label='MPI')
rects8 = ax.bar(x + 15*width/2, MRI, width, edgecolor='black', label='MRI')

# Fix the x-axes.
lab = ['residuals']
ax.set_xticks(x + 7*width / 2)
ax.set_xticklabels(lab,fontsize=18)
plt.ylabel('Explained Variance (%)',fontsize=18)
#ax.legend()


# create a function
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects0)
autolabel(rects1)
autolabel(rects2)
autolabel(rects3)
autolabel(rects4)
autolabel(rects5)
autolabel(rects6)
autolabel(rects7)
autolabel(rects8)
fig.tight_layout()
#plt.savefig(datadir + 'residuals_explained_variance_gcms_future[1].png',bbox_inches='tight', dpi = 600) 

In [ ]:
import seaborn as sns

In [ ]:
def grayify_cmap(cmap):
    """Return a grayscale version of the colormap"""
    cmap = plt.cm.get_cmap(cmap)
    colors = cmap(np.arange(cmap.N))
    
    # convert RGBA to perceived greyscale luminance
    # cf. http://alienryderflex.com/hsp.html
    RGB_weight = [0.299, 0.587, 0.114]
    luminance = np.sqrt(np.dot(colors[:, :3] ** 2, RGB_weight))
    colors[:, :3] = luminance[:, np.newaxis]
    
    return cmap.from_list(cmap.name + "_grayscale", colors, cmap.N)

### heatmap for era-interim SOM

In [ ]:
htmp = pd.read_csv('/terra/users/csag/kwesi/research/test_freq_trendy.csv',names = ['seas','freq','trend','node'], header =0)
print(htmp.head())

In [ ]:
#select season of interest
djf = htmp.query("seas in ['DJF']")
djf.head()


#select values and reshape to fit the 4 by 3 matrix for heatmap
djf_arr = np.array(djf.iloc[:,1]).reshape(3,4)
DF_djf = pd.DataFrame(djf_arr[:,:])
#print(DF_djf)


#make a dictionary to force plot in a strict order...'sorted' failed to do that huh!
headers_x = {1:'9',2:'10',3:'11',4:'12'}

xlist =[]
for key, value in headers_x.items():
    xlist.append(value)
    
headers_y = {1: '1', 2: '5', 3: '9'}

ylist =[]
for key, value in headers_y.items():
    ylist.append(value)

    
#plot the heatmap    
fig, ax = plt.subplots(figsize=(8,5))
fig.subplots_adjust(bottom=0.25,left=0.25) # make room for labels
import matplotlib.colors as colors

mmin = 1
mmax = 20

bounds = np.linspace(1, 20,40)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)


heatmap = ax.pcolor(DF_djf,edgecolors='k',norm=norm,cmap = 'Greys',vmin=mmin,vmax=mmax, linewidth=0.5) #norm=norm,grayify_cmap('bwr')
#sns.heatmap(DF_djf,cmap="YlGnBu")
#cbar = plt.colorbar(heatmap,ticks=np.arange(1,22,2))  

# Loop over data dimensions and create text annotations
data = np.array(DF_djf)
data_rav = data.ravel()
for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        if data_rav[0] < 0.05:
            plt.plot(DF_djf + 0.5, DF_djf + 0.5, marker='o', markersize=2, color="k")
        plt.text(x + 0.5, y + 0.5, '%.0f' % data[y, x],
                 horizontalalignment='center',
                 verticalalignment='center', color="r",fontsize=15)

# Set ticks in center of cells
ax.set_xticks(np.arange(DF_djf.shape[1]) + 0.5, minor=False)
ax.set_yticks(np.arange(DF_djf.shape[0]) + 0.5, minor=False)

# want a more natural, table-like display
ax.invert_yaxis()
#ax.xaxis.tick_top()


# Rotate the xlabels. Set both x and y labels to headers[0:]
ax.set_xticklabels(xlist[0:],fontsize=12) #,rotation=90
ax.set_yticklabels(ylist[0:],fontsize=12)
#ax.set_xlabel('SOM nodes',fontsize=15)
#ax.set_ylabel('SOM nodes',fontsize=15) 

#saving plot
plt.savefig('/home/kwesi/terra/research/paper2/plots/SOM_reanalysis_heatmap_djf_P3a.png',bbox_inches='tight', dpi = 600)
plt.show()

In [ ]:
#select season of interest
jja = htmp.query("seas in ['JJA']")
jja.head()


#select values and reshape to fit the 4 by 3 matrix for heatmap
jja_arr = np.array(jja.iloc[:,1]).reshape(3,4)
DF_jja = pd.DataFrame(jja_arr[:,:])
#print(DF_jja)


#make a dictionary to force plot in a strict order...'sorted' failed to do that huh!
headers_x = {1:'9',2:'10',3:'11',4:'12'}

xlist =[]
for key, value in headers_x.items():
    xlist.append(value)
    
headers_y = {1: '1', 2: '5', 3: '9'}

ylist =[]
for key, value in headers_y.items():
    ylist.append(value)

    
#plot the heatmap    
fig, ax = plt.subplots(figsize=(8,5))
fig.subplots_adjust(bottom=0.25,left=0.25) # make room for labels
import matplotlib.colors as colors

mmin = 1
mmax = 20

bounds = np.linspace(1, 20,40)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)


heatmap = ax.pcolor(DF_jja,edgecolors='k',norm=norm,cmap = 'Greys',vmin=mmin,vmax=mmax, linewidth=0.5) #norm=norm,grayify_cmap('bwr')
#sns.heatmap(DF_djf,cmap="YlGnBu")
#cbar = plt.colorbar(heatmap,ticks=np.arange(1,22,2))  

# Loop over data dimensions and create text annotations
data = np.array(DF_jja)
data_rav = data.ravel()
for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        if data_rav[0] < 0.05:
            plt.plot(DF_jja + 0.5, DF_jja + 0.5, marker='o', markersize=2, color="k")
        plt.text(x + 0.5, y + 0.5, '%.0f' % data[y, x],
                 horizontalalignment='center',
                 verticalalignment='center', color="red",fontsize=15)

# Set ticks in center of cells
ax.set_xticks(np.arange(DF_jja.shape[1]) + 0.5, minor=False)
ax.set_yticks(np.arange(DF_jja.shape[0]) + 0.5, minor=False)

# want a more natural, table-like display
ax.invert_yaxis()
#ax.xaxis.tick_top()

# Rotate the xlabels. Set both x and y labels to headers[0:]
ax.set_xticklabels(xlist[0:],fontsize=12) #,rotation=90
ax.set_yticklabels(ylist[0:],fontsize=12)
#ax.set_xlabel('SOM nodes',fontsize=15)
#ax.set_ylabel('SOM nodes',fontsize=15) 

#saving plot
plt.savefig('/home/kwesi/terra/research/paper2/plots/SOM_reanalysis_heatmap_jja_P3a.png',bbox_inches='tight', dpi = 600)
plt.show()

#### other ways of calculating explained variance===>there are some variations in there but most agree



In [ ]:
expl_var_pca = np.var(IPSL_scores_x, axis=0) / np.sum(np.var(IPSL_scores_x, axis=0))
print('explained variance pca: ', 100*expl_var_pca)

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

# some sample data
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000)).cumsum()

#plot the time series
ts.plot(style='k--')

# calculate a 60 day rolling mean and plot
ts.rolling(window=60).mean().plot(style='k')

# add the 20 day rolling standard deviation:
ts.rolling(window=20).std().plot(style='b')

#### alternatively;
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 9
plt.rcParams['figure.dpi'] = 130


ERAInterim = [30.0, 19.80, 12.60]
CanESM = [17.41,  21.65,  20.78]
CNRM  =  [15.50,  23.88,  21.56]
GFDL  =  [18.50,  21.17,  19.13]
HadGEM = [20.21,  18.11,  17.74]
IPSL  =  [17.66,  19.09,  23.56]
MIROC =  [19.98,  18.82,  21.30]
MPI   =  [20.92,  19.69,  16.65]
MRI   =  [16.81,  23.25,  19.35]



#create a custom colormap python
tableau_20 =[(31,119,180),(174,199,232),(255,127,14),(255,187,120),
            (44,160,44),(152,223,138),(214,39,40),(255,152,150),
            (148,103,189),(197,176,213),(140,86,75),(196,156,148),
            (227,119,194),(247,182,210),(127,127,127),(199,199,199),
            (188,189,34),(219,219,141),(23,190,207),(158,218,229)]

#           [steelblue,lightsteelblue,darkorange,peach(sandybrown),
#           green,lightgreen,crimsonred,lightcoral,
#           mediumpurple,palepurple,brown,darksalmon,
#           violet,pink,dimgrey,darkgrey,
#           olive,palegoldenrod,lightseagreen,powderblue]

#scaling above RBG values to [0,1] range, which is Matplotlib acceptable format:
for i in range(len(tableau_20)):
    r, g, b = tableau_20[i]
    tableau_20[i] = (r/255., g/255., b/255.)
    #plt.plot(tableau_20[i])
    #plt.pcolor(tableau_20)

labels = ['CanESM', 'CNRM', 'GFDL', 'HadGEM', 'IPSL', 'MIROC', 'MPI', 'MRI']



# set width of bar
barWidth = 0.1

# Set position of bar on X axis
r0 = np.arange(len(ERAInterim))
r1 = [x + barWidth for x in r0]
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
r5 = [x + barWidth for x in r4] 
r6 = [x + barWidth for x in r5]
r7 = [x + barWidth for x in r6]
r8 = [x + barWidth for x in r7]
 
# Make the plot
#3,5/6,7,8,9/10,12,15,17
plt.bar(r0, ERAInterim, color=tableau_20[1], width=barWidth, edgecolor='black', label='ERA-Interim')
plt.bar(r1, CanESM, color=tableau_20[3], width=barWidth, edgecolor='black', label='CanESM')
plt.bar(r2, CNRM, color=tableau_20[5], width=barWidth, edgecolor='black', label='CNRM')
plt.bar(r3, GFDL, color=tableau_20[6], width=barWidth, edgecolor='black', label='GFDL')
plt.bar(r4, HadGEM, color=tableau_20[8], width=barWidth, edgecolor='black', label='HadGEM2')
plt.bar(r5, IPSL, color=tableau_20[2], width=barWidth, edgecolor='black', label='IPSL')
plt.bar(r6, MIROC, color=tableau_20[10], width=barWidth, edgecolor='black', label='MIROC')
plt.bar(r7, MPI, color=tableau_20[7], width=barWidth, edgecolor='black', label='MPI')
plt.bar(r8, MRI, color=tableau_20[15], width=barWidth, edgecolor='black', label='MRI')



# Add xticks on the middle of the group bars
plt.xlabel('Components', fontweight='bold')
plt.xticks([r + 4.5*barWidth for r in range(len(CanESM))], ['RC1', 'RC2', 'RC3'])
 
# Create legend & Show graphic
plt.legend(loc='upper right')
#plt.show()